# Множественная проверка гипотез: разбор кейса

У вас есть такие данные о результативности штрафных бросков игроков NBA.

Каждый сезон NBA так же, как и в других видах спорта, проходит два этапа: регулярный сезон и плей-офф, когда играют на вылет. Понятно, что второй этап для всех более волнительный, и результативность игроков может отличаться. Мы попробуем проверить гипотезу о том, что этот этап сезона никак не влияет на результативность штрафных бросков игроков, и посчитаем разными методами количество игроков, у которых такое влияние всё-таки отмечается.

In [1]:
# Для начала нам надо посчитать данные:
import pandas as pd
import numpy as np

data = pd.read_csv('free_throws.csv')

In [2]:
# Посмотрим на формат:
data.head()

,end_result,game,game_id,period,play,player,playoffs,score,season,shot_made,time
0,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum makes free throw 1 of 2,Andrew Bynum,regular,0 - 1,2006 - 2007,1,11:45
1,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum makes free throw 2 of 2,Andrew Bynum,regular,0 - 2,2006 - 2007,1,11:45
2,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum makes free throw 1 of 2,Andrew Bynum,regular,18 - 12,2006 - 2007,1,7:26
3,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum misses free throw 2 of 2,Andrew Bynum,regular,18 - 12,2006 - 2007,0,7:26
4,106 - 114,PHX - LAL,261031013.0,1.0,Shawn Marion makes free throw 1 of 1,Shawn Marion,regular,21 - 12,2006 - 2007,1,7:18


Здесь нас интересуют следующие колонки:

* player — имя игрока
* playoffs — этап сезона NBA (regular/playoffs)
* shot_made — попал или не попал игрок штрафной бросок

Теперь будем для каждого игрока считать его среднюю результативность (долю попаданий) во время регулярного чемпионата и плей-офф, p_value, полученное через z-test, и запишем это в новый датафрейм. Z-test используем потому, что в данном случае наша задача является аналогом конверсии, например, в интернет-магазине, то есть мы имеем дело с распределением Бернулли.

Также мы оставим только тех игроков, у которых есть хотя бы по 30 бросков в регулярном сезоне и плей-офф. Это делаем для более корректной оценки, чтобы не считать тех игроков, у которых слишком мало бросков. Цифра 30 исходит из закона больших чисел, если коротко — такого количества наблюдений достаточно, чтобы довольно точно аппроксимировать нормальное распределение.

In [3]:
from statsmodels.stats.weightstats import ztest

new_df = {
    "player": [],
    "regular_mean": [],
    "playoff_mean": [],
    "p_value": []
}

for player, group in data.groupby("player"):
    regular_shots = group[group["playoffs"] == "regular"]["shot_made"].values
    playoff_shots = group[group["playoffs"] == "playoffs"]["shot_made"].values
    
    if len(regular_shots) < 30 or len(playoff_shots) < 30:
        continue
        
    statistic, p_value = ztest(regular_shots, playoff_shots)

    new_df["player"].append(player)
    new_df["regular_mean"].append(np.mean(regular_shots))
    new_df["playoff_mean"].append(np.mean(playoff_shots))
    new_df["p_value"].append(p_value)
    
new_df = pd.DataFrame(new_df)

In [4]:
# Посмотрим на 10 случайных элементов, чтобы увидеть, что у нас получилось:
new_df.sample(10)

,player,regular_mean,playoff_mean,p_value
155,Larry Hughes,0.754293,0.739130,0.780036
9,Andre Drummond,0.379649,0.323529,0.505106
29,Brad Miller,0.824811,0.838710,0.840969
154,Lance Stephenson,0.691622,0.677419,0.784251
111,Jason Maxiell,0.571795,0.500000,0.259019
8,Andray Blatche,0.723482,0.733333,0.867416
258,Tyson Chandler,0.658744,0.637500,0.584566
212,Ray Allen,0.908102,0.882353,0.121483
66,DeShawn Stevenson,0.712821,0.756757,0.566205
43,Chauncey Billups,0.907998,0.860526,0.003849


In [5]:
# Теперь посчитаем количество игроков с p_value < 0.05.
# Так мы поймём, для скольких игроков мы бы отвергли нулевую гипотезу
# без поправок о множественной проверки гипотез.
new_df[new_df.p_value < 0.05].shape

(22, 4)

Расчёты готовы, и нам необходимо сравнить их с методами Бонферрони и Холма.

In [6]:
# Импортируем функцию для множественной проверки гипотез:
from statsmodels.stats.multitest import multipletests

In [7]:
multipletests(new_df.p_value, alpha=0.05, method='bonferroni')[0].sum()

1

In [8]:
multipletests(new_df.p_value, alpha=0.05, method='holm')[0].sum()

1

Видим, что в данной задаче два метода выдали одинаковое количество игроков, для которых стоит отвергнуть нулевую гипотезу. То есть тех, у кого с 95 % отличается результативность во время плей-офф и регулярного сезона.

Однако мы видим, насколько данное число отличается от того, что мы получили без поправок на множественную проверку гипотез. Так нам удалось кратно уменьшить вероятность ошибок первого рода.